In [1]:
import pandas as pd
import numpy as np
managers = pd.read_csv('/Users/vsoma_ogh6dsk/Downloads/List_of_all_managers.csv')
managers

,Manager,Seasons,G,Team,Wins,Losses
0,Scott Servais,2016–,"1,194",SEA,NaN,NaN
1,Lloyd McClendon,2014–2015,324,SEA,NaN,NaN
2,Eric Wedge,2011–2013,486,SEA,NaN,NaN
3,Daren Brown,2010,50,SEA,NaN,NaN
4,Don Wakamatsu,2009–2010,274,SEA,NaN,NaN
...,...,...,...,...,...,...
220,Ryne Sandberg†,2013–2015,278,PHI,119,159
221,Charlie Manuel§,2005–2013,1416,PHI,780,636
222,Gary Varsho,2004,2,PHI,1,1
223,Larry Bowa§[b],2001–2004,645,PHI,337,308


In [2]:
managers = managers.drop(columns = ['Wins', 'Losses'])
managers = managers.rename(columns = {"G": "Games", "Manager": "Name"})
managers

,Name,Seasons,Games,Team
0,Scott Servais,2016–,"1,194",SEA
1,Lloyd McClendon,2014–2015,324,SEA
2,Eric Wedge,2011–2013,486,SEA
3,Daren Brown,2010,50,SEA
4,Don Wakamatsu,2009–2010,274,SEA
...,...,...,...,...
220,Ryne Sandberg†,2013–2015,278,PHI
221,Charlie Manuel§,2005–2013,1416,PHI
222,Gary Varsho,2004,2,PHI
223,Larry Bowa§[b],2001–2004,645,PHI


In [3]:
print(managers.describe())
print(managers.dtypes)

              Name Seasons Games Team
count          225     225   225  225
unique         164     133   152   30
top     Bob Melvin    2002   324  MIA
freq             5       6    21   14
Name       object
Seasons    object
Games      object
Team       object
dtype: object


In [4]:
def games_as_int(games):
    '''Convert the comma separated strings of games to an integer'''
    return int(games.replace(',', ''))

managers = managers.assign(Games = managers.get('Games').apply(games_as_int))

In [5]:
def strip_symbols(manager_name): 
    '''Get rid of Wikipedia's symbols at the end of managers' names'''
    return manager_name.replace('[b]', '').replace('[c]', '').replace('[d]', '').replace('[e]', '').replace('(interim)', '').strip('*‡†§ ').strip()

managers = managers.assign(Name = managers.get('Name').apply(strip_symbols))

In [6]:
managers.get('Name').unique().shape[0]/managers.shape[0]

0.6711111111111111

In [7]:
import wptools

def get_year_from_timestamp(birth_timestamp: str):
    if (type(birth_timestamp) is list):
        # If multiple timestamps found, last one is accurate year of birth
        birth_timestamp = birth_timestamp[-1] 
    birth_timestamp = birth_timestamp.strip('+') # Get rid of starting +
    return int(str.split(birth_timestamp, '-')[0])

def get_manager_birth_year(manager_name: str):
    '''Gets managers birth year from online lookup. Works for most manager names, but does not work for some.
    If multiple people have same name, function crashes seeing a disambiguation page. Returns None in those scenarios.
    Otherwise, returns the year that a manager was born. Function takes about 1.5 seconds per entry.'''
    try:
        manager_page = wptools.page(manager_name, silent = True) # Gets the page with wikidata without outputting to cout
    except LookupError:
        print(f"Could not locate manager {manager_name}'s wikipedia page")
        return None
    # Get information from the correct key-value pair: wikidata, date of birth are dictionaries
    try:
        manager_birth_timestamp = (manager_page.get().data['wikidata']['date of birth (P569)'])
    except ValueError:
        print(f"Could not locate manager {manager_name}'s date of birth")
        return None
    return get_year_from_timestamp(manager_birth_timestamp)

# Test
print(get_manager_birth_year('Scott Servais'), get_manager_birth_year('Sandy Alomar Jr.'))

1967 1966


In [8]:
# These are the names to resolve managers to be
managers = managers.assign(wiki_names = managers.get('Name'))

In [9]:
# Rename all names that link to disambiguation pages to the accurate page.
managers.get('wiki_names').iloc[6] = 'John McLaren (baseball)'
managers.get('wiki_names').loc[31] = 'Brad Mills (manager)'
managers.get('wiki_names').loc[32] = 'Dave Clark (baseball)'
managers.get('wiki_names').loc[75] = 'Tom Kelly (baseball)'
managers.get('wiki_names').loc[82] = 'John Farrell (manager)'
managers.get('wiki_names').loc[88] = 'John Schneider (baseball)'
managers.get('wiki_names').loc[91] = 'John Farrell (manager)'
managers.get('wiki_names').loc[115] = 'Dave Roberts (outfielder)'
managers.get('wiki_names').loc[119] = 'Jim Tracy (baseball)'
managers.get('wiki_names').loc[125] = 'Andy Green (baseball)'
managers.get('wiki_names').loc[126] = 'Dave Roberts (outfielder)'
managers.get('wiki_names').loc[127] = 'Pat Murphy (baseball coach)'
managers.get('wiki_names').loc[132] = 'Jim Tracy (baseball)'
managers.get('wiki_names').loc[144] = 'David Bell (baseball)'
managers.get('wiki_names').loc[154] = 'David Ross (baseball)'
managers.get('wiki_names').loc[177] = 'John Russell (catcher)'
managers.get('wiki_names').loc[178] = 'Jim Tracy (baseball)'
managers.get('wiki_names').loc[186] = 'Dan Jennings (manager)'
managers.get('wiki_names').loc[192] = 'Edwin Rodríguez (baseball)'
managers.get('wiki_names').loc[198] = 'John Boles (baseball)'
managers.get('wiki_names').loc[199] = 'Carlos Mendoza (baseball coach)'
managers.get('wiki_names').loc[201] = 'Luis Rojas (baseball)'
managers.get('wiki_names').loc[210] = 'Matt Williams (third baseman)'
managers.get('wiki_names').loc[212] = 'John McLaren (baseball)'

/var/folders/xv/svgwynh921zf62sfypxdy8w00000gq/T/ipykernel_5863/3291986749.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  managers.get('wiki_names').iloc[6] = 'John McLaren (baseball)'
/var/folders/xv/svgwynh921zf62sfypxdy8w00000gq/T/ipykernel_5863/3291986749.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  managers.get('wiki_names').loc[31] = 'Brad Mills (manager)'
/var/folders/xv/svgwynh921zf62sfypxdy8w00000gq/T/ipykernel_5863/3291986749.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [10]:
managers = managers.assign(birth_year = managers.get('wiki_names').apply(get_manager_birth_year))
managers

Could not locate manager Mike Hargrove's date of birth
Could not locate manager Lou Piniella's date of birth
Could not locate manager Joe Girardi's date of birth
Could not locate manager Juan Samuel's date of birth
Could not locate manager Mike Hargrove's date of birth
Could not locate manager Lou Piniella's date of birth
Could not locate manager Dave Roberts (outfielder)'s date of birth
Could not locate manager Dave Roberts (outfielder)'s date of birth
Could not locate manager Lou Piniella's date of birth


In [ ]:
managers[managers.get('birth_year').isnull()]

In [ ]:
managers.get('birth_year').iloc[116] = 1961
managers.get('birth_year').iloc[187] = 1961
managers.get('birth_year').iloc[77] = 1964
managers.get('birth_year').iloc[194] = 1964
managers.get('birth_year').iloc[7] = 1949
managers.get('birth_year').iloc[103] = 1949
managers.get('birth_year').iloc[115] = 1972
managers.get('birth_year').iloc[126] = 1972

In [ ]:
managers = managers.assign(birth_year = managers.get('birth_year').apply(int))
managers

In [ ]:
managers = managers.rename(columns = {"birth_year": "Birth_Year", "wiki_names": "Wiki_Names"})

Let's see if these date of birth values make sense by making I few plots. If they don't, I likely made an error in my script.

In [ ]:
sorted_managers = managers.sort_values(by = 'Birth_Year', ascending = True).get(['Name', 'Birth_Year'])
min_birth = sorted_managers.iloc[0].get('Birth_Year')
min_birth_person = sorted_managers.iloc[0].get('Name')
max_birth = sorted_managers.iloc[-1].get('Birth_Year')
max_birth_person = sorted_managers.iloc[-1].get('Name')
print(f"Earliest birth year is {min_birth_person} in {min_birth}. Latest birth is {max_birth_person} in {max_birth}")

In [ ]:
import matplotlib.pyplot as plt
managers.plot(kind = 'hist', y = 'Birth_Year', ec = 'w', xlabel = 'Birth Year by manager who coached from 2000-present');
managers.plot(kind = 'hist', y = 'Games', ec = 'w', xlabel = 'Number of Games by manager who coached from 2000-present');

In [ ]:
csv_data = managers.to_csv(index = False)
file_path = 'managers_with_birth.csv'
with open(file_path, 'w') as f:
    f.write(csv_data)
